In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVR

## Importing Data and DateTime index preparation

In [ ]:
df = pd.read_csv("data/bicikelj_train.csv")
# Convert the "timespamp" colum to a datetime object and set it as the index
df = df.set_index("timestamp")
df.index = pd.to_datetime(df.index)

# Count the number of duplicate index values
display(df.index.duplicated().sum())

display(df.head())
display(df.info())

## Creating a DataFrame for each station

In [ ]:
# Split the dataframe into multiple dataframes, each with one station
df_list = [df[[station]] for station in df.columns]
df_list[0].head()

### Data distribution of one station

In [ ]:
ig, ax = plt.subplots(figsize=(20, 5))
df_list[0].plot(ax=ax, label='Training Set', title=f"Visualization for {df_list[0].columns[0]} station", style=".")
plt.show()

## Time Series Cross Validation Example and Visualization

In [ ]:
tss = TimeSeriesSplit(n_splits=10, gap=(60 // 5))
df_list = [station.sort_index() for station in df_list]

In [ ]:
fig, axs = plt.subplots(10, 1, figsize=(15, 15), sharex=True)

fold = 0
for train_idx, val_idx in tss.split(df_list[0]):
    train = df_list[0].iloc[train_idx]
    test = df_list[0].iloc[val_idx]
    train[df_list[0].columns[0]].plot(ax=axs[fold],
                          label='Training Set',
                          title=f'Data Train/Test Split Fold {fold}')
    test[df_list[0].columns[0]].plot(ax=axs[fold],
                         label='Test Set')
    axs[fold].axvline(test.index.min(), color='black', ls='--')
    fold += 1
plt.show()

# Feature creation

## Features created from datetime index (hour, dayofweek, month, dayofyear, dayofmonth)

In [ ]:
def create_features(station):
    """
    Create time series features based on time series index.
    """
    station = station.copy()
    station['hour'] = station.index.hour
    station['dayofweek'] = station.index.dayofweek
    station['month'] = station.index.month
    station['dayofyear'] = station.index.dayofyear
    station['dayofmonth'] = station.index.day
    return station

df_list = [create_features(station) for station in df_list]
df_list[0].head()

## Lag features (60 and 120 min lag)

In [ ]:
def add_lags(station):
    station = station.copy()
    # Create 60, 90 and 120 minutes lags
    # lags = [60, 90, 120]
    lags = [60, 120]
    for lag in lags:
        df_lag = station.copy()
        # Drop all but the target column
        df_lag = df_lag[[station.columns[0]]]
        # Subtract 60 minutes from the index
        df_lag.index = df_lag.index + pd.Timedelta(minutes=lag)
        # Rename the column to "lag_60"
        df_lag.columns = [f"lag_{lag}"]
        # Merge the dataframe with the lagged dataframe
        station = pd.merge_asof(
            station,
            df_lag,
            left_index=True,
            right_index=True,
            direction="nearest",
            tolerance=pd.Timedelta("15m"),
        )
    return station

df_list = [add_lags(station) for station in df_list]
display(df_list[0].tail())
display(df_list[0].isna().sum())

In [ ]:
df_list[0].to_csv("data/other/processed.csv")

## School day feature (0 for holidays and weekends, 1 for school days)

In [ ]:
def add_school_days(station):
    # Add vacation feature which is 1 if the day is a school vacation day and 0 otherwise
    # School days are from 1.9.2022 to 31.10.2022 without weekends
    station["vacation"] = 0
    station.loc[(station.index >= "2022-09-01") & (station.index <= "2022-10-31") & (station.index.dayofweek < 5), "vacation"] = 1
    return station

df_list = [add_school_days(station) for station in df_list]
display(df_list[0].tail())

## Weather feature (Amount of rain in mm)

In [ ]:
weather = pd.read_csv("data/extra/padavine_polurno.csv")
weather = weather.drop(columns=["station_id", "station_name"])
weather["datum"] = pd.to_datetime(weather["datum"])
weather = weather.set_index("datum")
weather.info()

def add_weather(df, weather):
    df = df.copy()
    weather = weather.copy()
    # Add weather features by matrhing the Date, Hour of the day and the station
    return pd.merge_asof(
        df,
        weather,
        left_index=True,
        right_index=True,
        direction="nearest",
    )

df_list = [add_weather(station, weather) for station in df_list]
display(df_list[0].tail())

# Free spaces feature

In [ ]:
metadata = pd.read_csv("data/extra/bicikelj_metadata.csv")

def free_spaces(station):
    station = station.copy()
    # Get the total number of spaces for the station
    station_metadata = metadata[metadata["postaja"] == station.columns[0]]
    total_space = station_metadata["total_space"].values[0]
    lags = [60, 120]
    for lag in lags:
        station_lag = station.copy()
        # Drop all but the target column
        station_lag = station_lag[[station.columns[0]]]
        # Subtract the number of occupied spaces from the total number of spaces
        station_lag[station.columns[0]] = total_space - station_lag[station.columns[0]]
        # Subtract 60 minutes from the index
        station_lag.index = station_lag.index + pd.Timedelta(minutes=lag)
        # Rename the column to "lag_60"
        station_lag.columns = [f"free_space_lag_{lag}"]
        # Merge the dataframe with the lagged dataframe
        station = pd.merge_asof(
            station,
            station_lag,
            left_index=True,
            right_index=True,
            direction="nearest",
            tolerance=pd.Timedelta("15m"),
        )
    return station

df_list = [free_spaces(station) for station in df_list]
display(df_list[0].tail())

# Closest stations feature

In [ ]:
def add_closest_station_lags(station1, station2, index):
    station1 = station1.copy()
    station2 = station2.copy()
    # Create 60, 90 and 120 minutes lags
    # lags = [60, 90, 120]
    lags = [60, 120]
    for lag in lags:
        station2_lag = station2.copy()
        # Drop all but the target column
        station2_lag = station2_lag[[station2.columns[0]]]
        # Subtract 60 minutes from the index
        station2_lag.index = station2_lag.index + pd.Timedelta(minutes=lag)
        # Rename the column to "lag_60"
        station2_lag.columns = [f"station_{index}_lag_{lag}"]
        # Merge the dataframe with the lagged dataframe
        station1 = pd.merge_asof(
            station1,
            station2_lag,
            left_index=True,
            right_index=True,
            direction="nearest",
            tolerance=pd.Timedelta("15m"),
        )
    return station1





number_of_closest_stations = 5
def add_n_closest(station, df, metadata, n=5):
    station = station.copy()
    metadata = metadata.copy()
    metadata = metadata.copy()
    # Get the row where the "postaja" column matches the station name
    station_metadata = metadata[metadata["postaja"] == station.columns[0]]
    longitude, latitude = station_metadata["geo-visina"].values[0], station_metadata["geo-sirina"].values[0]

    # Add the distance from the station to every other station
    metadata["razdralja"] = np.sqrt((metadata["geo-visina"] - longitude) ** 2 + (metadata["geo-sirina"] - latitude) ** 2)
    # Sort the dataframe by the distance
    closest_stations_names = metadata.sort_values("razdralja").iloc[1:n + 1]["postaja"].values
    # Get the dataframes of the closest stations
    closest_stations_df = [pd.DataFrame(df[df.columns[0]]) for df in df if df.columns[0] in closest_stations_names]
    # Add the closest station lags to the dataframe
    for index, closest_station in enumerate(closest_stations_df):
        station = add_closest_station_lags(station, closest_station, index + 1)
    return station

df_list = [add_n_closest(station, df_list, metadata, number_of_closest_stations) for station in df_list]




# Preprocessing (OneHotEncoding)

In [ ]:
# OneHot encode the categorical features
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
# Define the categorical features
categoricals = ["hour", "dayofweek", "month"]
# Fit the encoder on the categorical features only
encoder.fit(df_list[0][categoricals])
def one_hot_encode(df, encoder=encoder, features=categoricals):
    df = df.copy()
    # Encode the categorical features
    encoded = pd.DataFrame(encoder.transform(df[features]))
    encoded.columns = encoder.get_feature_names_out(features)
    encoded.index = df.index
    # Concatenate the original dataframe and the encoded dataframe
    df = pd.concat([df, encoded], axis=1)
    # Drop the original categorical columns
    df = df.drop(features, axis=1)
    return df

In [ ]:
# Use the one hot encoder on all dataframes
df_list = [one_hot_encode(station) for station in df_list]

display(df_list[0].tail())

# Split the data into 60 min and 120 min lag

In [ ]:
# # From the dataframes make two dataframes, one with all the features and 60 minutes lag and one with 120 minutes lag
df_list_60 = [station.drop(columns=["lag_120", "free_space_lag_120"]) for station in df_list]
for i in range(number_of_closest_stations):
    df_list_60 = [station.drop(columns=[f"station_{i+1}_lag_120"]) for station in df_list_60]
df_list_120 = [station.drop(columns=["lag_60", "free_space_lag_60"]) for station in df_list]
for i in range(number_of_closest_stations):
    df_list_120 = [station.drop(columns=[f"station_{i+1}_lag_60"]) for station in df_list_120]

# Model Testing With Cross Validation

In [ ]:
def cross_validation_testing(df_list, params=None):
        tss = TimeSeriesSplit(n_splits=5, gap=(60 // 5))
        df_list = [station.sort_index() for station in df_list]
        scores_list = []
        preds_list = []
        for station in df_list:
                station.dropna(inplace=True)
                preds = []
                scores = []
                for train_idx, val_idx in tss.split(station):
                        train = station.iloc[train_idx]
                        test = station.iloc[val_idx]

                        FEATURES = train.columns[1:]
                        TARGET = station.columns[0]

                        X_train = train[FEATURES]
                        y_train = train[TARGET]

                        X_test = test[FEATURES]
                        y_test = test[TARGET]

                        model = SVR()
                        
                        model.fit(X_train, y_train)

                        y_pred = model.predict(X_test)
                        preds.append(y_pred)
                        score = mean_absolute_error(y_test, y_pred)
                        scores.append(score)
                
                preds_list.append(preds)
                scores_list.append(scores)

                print(f'{station.columns[0]}: Score across folds {np.mean(scores):0.4f}')
                # print(f'{station.columns[0]}: Fold scores:{scores}')
        return scores_list, preds_list

### Testing the 60 min lag model before hyperparameter tuning

In [ ]:
scores_list_60, preds_list_60 = cross_validation_testing(df_list_60)
print(f'Average Score: {np.mean(scores_list_60):0.4f}')

### Testing the 120 min lag model before hyperparameter tuning

In [ ]:
scores_list_120, preds_list_120 = cross_validation_testing(df_list_120)
print(f'Average Score: {np.mean(scores_list_120):0.4f}')

# Hyperparameter tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
def hyperparameter_tuning(df_station):
    df_station.dropna(inplace=True)
    tss = TimeSeriesSplit(n_splits=5, gap=(60 // 5))
    df_station = df_station.sort_index()
    params = {
        'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
        'gamma': ['scale', 'auto'],
        'tol': [1e-3, 1e-4, 1e-2],
        'C': [0.1, 1.0, 10.0, 100.0],
        'epsilon': [0.1, 0.2, 0.05],
        'shrinking': [True, False],
            }
    
    FEATURES = df_station.columns[1:]
    TARGET = df_station.columns[0]
    SVR_reg = SVR()
    # SVR_grid = GridSearchCV(SVR_reg, params, cv=tss, verbose=2, n_jobs=-1, scoring='neg_mean_absolute_error')
    SVR_grid = RandomizedSearchCV(SVR_reg, params, cv=tss, verbose=2, n_jobs=-1, scoring='neg_mean_absolute_error', n_iter=10, random_state=42)
    SVR_grid.fit(df_station[FEATURES], df_station[TARGET])
    print(f"{df_station.columns[0]}: {SVR_grid.best_params_}")
    return SVR_grid

In [ ]:
model_tuning_60 = [hyperparameter_tuning(station) for station in df_list_60]

In [ ]:
model_tuning_120 = [hyperparameter_tuning(station) for station in df_list_120]

In [ ]:
# Save the best parameters for each station in a dictionary
best_params_60 = {station.columns[0]: model.best_params_ for station, model in zip(df_list_60, model_tuning_60)}
best_params_120 = {station.columns[0]: model.best_params_ for station, model in zip(df_list_120, model_tuning_120)}

In [ ]:
# Save the best hyperparameters
# Save the best parameters in a json file
import pickle
with open("hyperparameters/best_SVR_params_60.pkl", "wb") as f:
    pickle.dump(best_params_60, f)

with open("hyperparameters/best_SVR_params_120.pkl", "wb") as f:
    pickle.dump(best_params_120, f)

In [ ]:
# Read the best parameters from the pkl file
import pickle
with open("hyperparameters/best_SVR_params_60.pkl", "rb") as f:
    best_params_60 = pickle.load(f)

with open("hyperparameters/best_SVR_params_120.pkl", "rb") as f:
    best_params_120 = pickle.load(f)

## Training the model on the whole dataset (One for each station)

In [ ]:
def train_models(df_list, params):
    models = []
    for station in df_list:
        station.dropna(inplace=True)

        FEATURES = station.columns[1:]
        TARGET = station.columns[0]
        
        X_all = station[FEATURES]
        y_all = station[TARGET]
        
        reg = SVR(**params[station.columns[0]])
        reg.fit(X_all, y_all)
        
        models.append(reg)
    return models

In [ ]:
models_60 = train_models(df_list_60, best_params_60)
models_120 = train_models(df_list_120, best_params_120)

## Preparing the test data for prediction

In [ ]:
# Reading and preparing the test data
df_test = pd.read_csv("data/bicikelj_test.csv")
df_test = df_test.set_index("timestamp")
df_test.index = pd.to_datetime(df_test.index)
df_test = df_test.sort_index()

# Create a list of dataframes, one for each station for the test and train set
df_train_list = [df[[station]] for station in df.columns]
df_test_list = [df_test[[station]] for station in df_test.columns]

# Add the "Test" column to the dataframes
df_train_list = [station.assign(Test=False) for station in df_train_list]
df_test_list = [station.assign(Test=True) for station in df_test_list]

test_df_list = []
for train, test in zip(df_train_list, df_test_list):
        # Concatenate the two dataframes
        train_n_test = pd.concat([train, test], axis=0)
        train_n_test = train_n_test.sort_index()
        # Add the lags and datetime features
        train_n_test = create_features(train_n_test)
        train_n_test = add_lags(train_n_test)
        train_n_test = add_school_days(train_n_test)
        train_n_test = add_weather(train_n_test, weather)
        train_n_test = free_spaces(train_n_test)
        train_n_test = add_n_closest(train_n_test, df_list, metadata, n=5)
        # One hot encode the categorical features
        train_n_test = one_hot_encode(train_n_test)
        # Select only the rows which are in the test set
        station_test = train_n_test[train_n_test["Test"] == True]
        # Drop the "Test" column
        station_test = station_test.drop("Test", axis=1)
        # Add the dataframe to the list
        test_df_list.append(station_test)

display(test_df_list[0].head())
display(test_df_list[0].isna().sum())

In [ ]:
# Make a list of dataframes from rows where lag_60 is not null
df_list_60 = [station[station["lag_60"].notnull()] for station in test_df_list]
# Remove the lag_120 columns 
df_list_60 = [station.drop("lag_120", axis=1) for station in df_list_60]
df_list_60 = [station.drop("free_space_lag_120", axis=1) for station in df_list_60]
for i in range(number_of_closest_stations):
    df_list_60 = [station.drop(f"station_{i+1}_lag_120", axis=1) for station in df_list_60]

# Make a list of dataframes from rows where lag_60 is null
df_list_120 = [station[station["lag_60"].isnull()] for station in test_df_list]
# Remove the lag_60 columns
df_list_120 = [station.drop("lag_60", axis=1) for station in df_list_120]
df_list_120 = [station.drop("free_space_lag_60", axis=1) for station in df_list_120]
for i in range(number_of_closest_stations):
    df_list_120 = [station.drop(f"station_{i+1}_lag_60", axis=1) for station in df_list_120]

## Predicting the test data

In [37]:
def make_predictions(models, df_list):
    preds_list = []
    for model, station in zip(models, df_list):
        # Get the predictions for the station
        pred = model.predict(station[station.columns[1:]])
        # Transform the predictions to integers
        # pred = pred.round(0).astype(np.int64)
        # Add the predictions to the list
        preds_list.append(pred)
    return preds_list

In [38]:
# Make the predictions
preds_list_60 = make_predictions(models_60, df_list_60)
preds_list_120 = make_predictions(models_120, df_list_120)

In [39]:
# Combine the predictions for 60 and 120 minutes into one list
preds_list_merged = []
for pred_60, pred_120 in zip(preds_list_60, preds_list_120):
    pred = []
    for i in range(len(pred_120)):
        pred.append(pred_60[i])
        pred.append(pred_120[i])
    preds_list_merged.append(pred)    

In [40]:
# Add the predictions to the test set
predicted = pd.read_csv("data/bicikelj_test.csv")
stations = predicted.columns[1:]
for station, pred in zip(stations, preds_list_merged):
        predicted[station] = pred

In [41]:
# Change every negative value to 0
for station in predicted.columns[1:]:
    predicted[station] = predicted[station].apply(lambda x: 0 if x < 0 else x)

# From each row of the metadata dataframe, get the "postaja" and the "total_space" and add them to a dictionary
capacity_dict = {}
for index, row in metadata.iterrows():
    capacity_dict[row["postaja"]] = row["total_space"]

# For each station check if the predicted number of bikes is greater than the capacity and if so, set it to the capacity
for station in predicted.columns[1:]:
    predicted[station] = predicted[station].apply(lambda x: capacity_dict[station] if x > capacity_dict[station] else x)

In [42]:

predicted.to_csv("data/predictions/predicted_SVR_final.csv", index=False)

In [43]:
# import pickle
# with open("models/models_SVR_60.pkl", "wb") as f:
#     pickle.dump(models_60, f)

# with open("models/models_SVR_120.pkl", "wb") as f:
#     pickle.dump(models_120, f)